# <b>Riding The Data Trail</b>
## Notebook 3: Modeling & Hyperparameter Tuning

# Table of contents
* [Import Libraries](#import-libraries)
* [Import csv](#import-csv)
* [Pre-Modeling Prep](#pre-modeling-prep)     
* [BERTopic](#bertopic)  

# Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from scipy.special import inv_boxcox
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [2]:
pd.set_option('display.max_columns', 50)

# Import csv

In [3]:
current_directory = os.getcwd()
file_path = os.path.join(current_directory, '../data', 'cleaned_df.csv')

df = pd.read_csv(file_path)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1443 entries, 0 to 1442
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   no_of_likes            1443 non-null   int64  
 1   no_of_images           1443 non-null   int64  
 2   title                  1443 non-null   object 
 3   listing_price          1443 non-null   object 
 4   item_condition         1443 non-null   object 
 5   deal_method            1443 non-null   object 
 6   post_date              1443 non-null   object 
 7   category_type          1443 non-null   object 
 8   post_type              1443 non-null   object 
 9   condition_subtext      1443 non-null   object 
 10  listing_description    1419 non-null   object 
 11  mailing_option         1443 non-null   object 
 12  delivery_options       116 non-null    object 
 13  mail_speed             116 non-null    object 
 14  meetup_option          1443 non-null   object 
 15  meet

# Pre-Modeling Prep

In [5]:
model_df=df[['no_of_likes',	'no_of_images',	'title', 'listing_price', 'item_condition',	'deal_method', 'post_date',	'category_type', 
                 'post_type', 'condition_subtext',	'mailing_option', 'meetup_option', 'meetup_location', 'seller_id', 'seller_join_date', 
                 'seller_response', 'seller_verif', 'verified_by_email', 'verified_by_facebook', 'verified_by_mobile', 
                 'seller_stars_rating',	'reviews_of_seller', 'brands', 'current_listing_price', 'len_posts', 
                 'post_word_count',	'num_emojis', 'lemma_posts']]

## BERTopic

In [6]:
# Display output
from IPython.display import display

# Data processing
import pandas as pd
import numpy as np

# Topic model
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic import BERTopic

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=60, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer()

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# All steps together
topic_model = BERTopic(
  embedding_model = embedding_model,    # Step 1 - Extract embeddings
  umap_model = umap_model,              # Step 2 - Reduce dimensionality
  hdbscan_model = hdbscan_model,        # Step 3 - Cluster reduced embeddings
  vectorizer_model = vectorizer_model,  # Step 4 - Tokenize topics
  ctfidf_model = ctfidf_model,          # Step 5 - Extract topic words
  calculate_probabilities = True       # Step 7 - Calculate probabilities of topics found in each document
)

# Run BERTopic model
topics, probabilities = topic_model.fit_transform(model_df['lemma_posts'])

# Get and save the list of topics
display(topic_model.get_topic_info())
# topic_model.get_topic_info().to_csv('topics.csv')

,Topic,Count,Name,Representation,Representative_Docs
0,-1,719,-1_bike_size_carbon_shimano,"[bike, size, carbon, shimano, frame, mm, road,...",[nego till let carbon road bike ready ride lik...
1,0,206,0_specialized_bike_mm_carbon,"[specialized, bike, mm, carbon, shimano, tarma...",[specialized s work tarmac sl6 ultralight size...
2,1,158,1_giant_tcr_shimano_advanced,"[giant, tcr, shimano, advanced, pro, size, car...",[giant tcr adv 2 disc road bike giant tcr adva...
3,2,109,2_merida_carbon_reacto_bike,"[merida, carbon, reacto, bike, ultegra, shiman...",[merida reacto carbon road bike size xs 47 shi...
4,3,88,3_trek_madone_bontrager_oclv,"[trek, madone, bontrager, oclv, bike, slr, siz...",[trek madone slr 7 di2 gen 7 quicksilver 56 sa...
5,4,82,4_pinarello_dogma_size_di2,"[pinarello, dogma, size, di2, dura, ace, f12, ...",[pinarello dogma f12 myway pinarello dogma f12...
6,5,81,5_canyon_cf_bike_ultimate,"[canyon, cf, bike, ultimate, endurace, aeroad,...",[canyon ultimate cf sl 9.0 carbon frame size s...


In [8]:
# Add topics and probabilities to the DataFrame
model_df['topic'] = topics
model_df['probability'] = probabilities.max(axis=1)  # Get the max probability for each row

# Get the topic representations
topic_info = topic_model.get_topic_info()

# # Create new columns for each of the top 10 words from each topic
# for i in range(10):
#     # Extract the ith word from the 'Representation' column
#     model_df[f'x{i+1}'] = topic_info['Representation'].apply(lambda words: words[i] if len(words) > i else None)

In [9]:
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,719,-1_bike_size_carbon_shimano,"[bike, size, carbon, shimano, frame, mm, road,...",[nego till let carbon road bike ready ride lik...
1,0,206,0_specialized_bike_mm_carbon,"[specialized, bike, mm, carbon, shimano, tarma...",[specialized s work tarmac sl6 ultralight size...
2,1,158,1_giant_tcr_shimano_advanced,"[giant, tcr, shimano, advanced, pro, size, car...",[giant tcr adv 2 disc road bike giant tcr adva...
3,2,109,2_merida_carbon_reacto_bike,"[merida, carbon, reacto, bike, ultegra, shiman...",[merida reacto carbon road bike size xs 47 shi...
4,3,88,3_trek_madone_bontrager_oclv,"[trek, madone, bontrager, oclv, bike, slr, siz...",[trek madone slr 7 di2 gen 7 quicksilver 56 sa...
5,4,82,4_pinarello_dogma_size_di2,"[pinarello, dogma, size, di2, dura, ace, f12, ...",[pinarello dogma f12 myway pinarello dogma f12...
6,5,81,5_canyon_cf_bike_ultimate,"[canyon, cf, bike, ultimate, endurace, aeroad,...",[canyon ultimate cf sl 9.0 carbon frame size s...


In [10]:
# Get the top 10 words from each topic
top_words = topic_info['Representation'].apply(lambda words: words[:10])

# Print the top words for each topic
for i, words in enumerate(top_words):
    print(f"Topic {i}: {', '.join(words)}")

Topic 0: bike, size, carbon, shimano, frame, mm, road, new, cm, speed
Topic 1: specialized, bike, mm, carbon, shimano, tarmac, size, saddle, speed, ultegra
Topic 2: giant, tcr, shimano, advanced, pro, size, carbon, disc, wheelset, advance
Topic 3: merida, carbon, reacto, bike, ultegra, shimano, size, scultura, frame, road
Topic 4: trek, madone, bontrager, oclv, bike, slr, size, aeolus, emonda, carbon
Topic 5: pinarello, dogma, size, di2, dura, ace, f12, bike, ultegra, wheelset
Topic 6: canyon, cf, bike, ultimate, endurace, aeroad, size, sl, shimano, new


In [11]:
# Create a dictionary mapping each topic to its top 10 words
top_words_dict = topic_info.set_index('Topic')['Representation'].apply(lambda words: words[:10]).to_dict()

# Map the 'topic' column to the top 10 words and split them into separate columns
model_df[['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10']] = model_df['topic'].map(top_words_dict).apply(pd.Series)

In [12]:
model_df.head(30)

,no_of_likes,no_of_images,title,listing_price,item_condition,deal_method,post_date,category_type,post_type,condition_subtext,mailing_option,meetup_option,meetup_location,seller_id,seller_join_date,seller_response,seller_verif,verified_by_email,verified_by_facebook,verified_by_mobile,seller_stars_rating,reviews_of_seller,brands,current_listing_price,len_posts,post_word_count,num_emojis,lemma_posts,topic,probability,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
0,4,6,"Specialized Allez Sport - Year 2020, Size 52","2,200",Lightly used,Meetup,2 days ago,Road Bikes,Posted,"Used with care. Flaws, if any, are barely noti...",N,Y,Bakau LRT Station (SE3),tcw.jason,Joined 6 years ago,Mostly Responsive,Verified,1.0,0.0,1.0,5.0,17,"Specialized, SPECIALIZED, SPECIALISED, Allez, ...",2200.0,481,85,0,specialized allez sport year 2020 size 52 2020...,0,0.191040,specialized,bike,mm,carbon,shimano,tarmac,size,saddle,speed,ultegra
1,14,3,Giant OCR,500,Well used,Meetup,1 week ago,Road Bikes,Posted,Has minor flaws or defects.,N,Y,Bendemeer Light,jktay,Joined 8 years ago,Very Responsive,Verified,1.0,1.0,1.0,5.0,7,Giant,500.0,62,11,0,giant ocr m size claris shifters fd rd mavic c...,1,0.779363,giant,tcr,shimano,advanced,pro,size,carbon,disc,wheelset,advance
2,9,7,Polygon Hybrid Bicycle,500,Lightly used,Mailing,1 week ago,Road Bikes,Posted,"Used with care. Flaws, if any, are barely noti...",Y,N,NaN,starfish100,Joined 7 years ago,Very Responsive,Verified,1.0,1.0,1.0,5.0,227,look,500.0,487,87,0,polygon hybrid bicycle lightly bike balcony cl...,-1,0.152828,bike,size,carbon,shimano,frame,mm,road,new,cm,speed
3,28,5,"Cervelo P5X tt bike, road bike triathlon bike ...","16,000",Like new,Meetup,23 hours ago,Road Bikes,Bumped,Used once or twice. As good as new.,N,Y,SingaporeAdmiralty,sdm.co,Joined 3 years ago,Very Responsive,Verified,1.0,1.0,1.0,4.9,36,"Cervelo, Cervelo, Cervelo",16000.0,739,132,0,cervelo p5x tt bike road bike triathlon bike p...,-1,0.149013,bike,size,carbon,shimano,frame,mm,road,new,cm,speed
4,1,7,WTS/WTT Scott Speedster 20 (Size 56),"2,000",Like new,Meetup,2 days ago,Road Bikes,Posted,Used once or twice. As good as new.,N,Y,Cashew MRT Station (DT2),rizeal,Joined 1 year ago,Very Responsive,Verified,1.0,0.0,1.0,5.0,10,Scott,2000.0,1237,186,0,wts wtt scott speedster 20 size 56 buy kh cycl...,-1,0.124765,bike,size,carbon,shimano,frame,mm,road,new,cm,speed
5,3,1,Giant TC ADVANCE 3 DISC,"1,600",Lightly used,Meetup,1 week ago,Road Bikes,Posted,"Used with care. Flaws, if any, are barely noti...",N,Y,Prestige heights,zhaoyichen82949,Joined 1 month ago,Not Responsive,Verified,1.0,0.0,1.0,0.0,0,Giant,1600.0,71,7,0,giant tc advance 3 disc 年初购入，因为要回，忍痛转让 尺寸m，附赠打...,1,0.543675,giant,tcr,shimano,advanced,pro,size,carbon,disc,wheelset,advance
6,5,1,Canyon Endurace CF SL 8.0 XS,"3,500",Lightly used,Meetup,2 days ago,Road Bikes,Posted,"Used with care. Flaws, if any, are barely noti...",N,Y,Livia,jeremiahheng,Joined 7 years ago,Very Responsive,Verified,1.0,1.0,1.0,5.0,22,"Canyon, Canyon",3500.0,468,81,0,canyon endurace cf sl 8.0 xs selling canyon en...,5,1.000000,canyon,cf,bike,ultimate,endurace,aeroad,size,sl,shimano,new
7,14,9,Canyon Ultimate CFR Size S - Titan Silver,"11,000 12,000",Lightly used,Meetup,2 months ago,Road Bikes,Posted,"Used with care. Flaws, if any, are barely noti...",N,Y,38 Anchorvale LaneRivercove residences,yao10,Joined 8 years ago,Very Responsive,Verified,1.0,0.0,1.0,4.9,72,"Canyon, Canyon, Canyon, Pinarello, Dogma, look",11000.0,1344,220,0,canyon ultimate cfr size s titan silver 2022 c...,5,1.000000,canyon,cf,bike,ultimate,endurace,aeroad,size,sl,shimano,new
8,115,4,Authentic Pinarello GAN RS Maglia Nera road bi...,"3,100",Lightly used,Meetup,5 days ago,Road Bikes,Bumped,"Used with care. Flaws, if any, are barely noti...",N,Y,Hougang Parkview,nitsbitsnwits,Joined 9 years ago,Very Responsive,Verified,1.0,1.0,1.0,5.0,465,Pinarello,3100.0,379,62,0,authentic pinarello gan rs maglia nera road bi...,4,1.000000,pinarello,dogma,size,di2,dura,ace,f12,bike,ult

In [13]:
low_model_df=model_df[model_df['current_listing_price']<=2400]

In [14]:
low_model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 674 entries, 0 to 1442
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   no_of_likes            674 non-null    int64  
 1   no_of_images           674 non-null    int64  
 2   title                  674 non-null    object 
 3   listing_price          674 non-null    object 
 4   item_condition         674 non-null    object 
 5   deal_method            674 non-null    object 
 6   post_date              674 non-null    object 
 7   category_type          674 non-null    object 
 8   post_type              674 non-null    object 
 9   condition_subtext      674 non-null    object 
 10  mailing_option         674 non-null    object 
 11  meetup_option          674 non-null    object 
 12  meetup_location        656 non-null    object 
 13  seller_id              674 non-null    object 
 14  seller_join_date       674 non-null    object 
 15  selle

In [15]:
# Feature Engineering
def price_cat(price):
    if price <= 600:
        return 0
    elif (price >600) & (price <= 1200):
        return 1
    else:
        return 2

low_model_df['price'] = low_model_df['current_listing_price'].apply(price_cat)


In [16]:
low_model_df

,no_of_likes,no_of_images,title,listing_price,item_condition,deal_method,post_date,category_type,post_type,condition_subtext,mailing_option,meetup_option,meetup_location,seller_id,seller_join_date,seller_response,seller_verif,verified_by_email,verified_by_facebook,verified_by_mobile,seller_stars_rating,reviews_of_seller,brands,current_listing_price,len_posts,post_word_count,num_emojis,lemma_posts,topic,probability,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,price
0,4,6,"Specialized Allez Sport - Year 2020, Size 52","2,200",Lightly used,Meetup,2 days ago,Road Bikes,Posted,"Used with care. Flaws, if any, are barely noti...",N,Y,Bakau LRT Station (SE3),tcw.jason,Joined 6 years ago,Mostly Responsive,Verified,1.0,0.0,1.0,5.0,17,"Specialized, SPECIALIZED, SPECIALISED, Allez, ...",2200.0,481,85,0,specialized allez sport year 2020 size 52 2020...,0,0.191040,specialized,bike,mm,carbon,shimano,tarmac,size,saddle,speed,ultegra,2
1,14,3,Giant OCR,500,Well used,Meetup,1 week ago,Road Bikes,Posted,Has minor flaws or defects.,N,Y,Bendemeer Light,jktay,Joined 8 years ago,Very Responsive,Verified,1.0,1.0,1.0,5.0,7,Giant,500.0,62,11,0,giant ocr m size claris shifters fd rd mavic c...,1,0.779363,giant,tcr,shimano,advanced,pro,size,carbon,disc,wheelset,advance,0
2,9,7,Polygon Hybrid Bicycle,500,Lightly used,Mailing,1 week ago,Road Bikes,Posted,"Used with care. Flaws, if any, are barely noti...",Y,N,NaN,starfish100,Joined 7 years ago,Very Responsive,Verified,1.0,1.0,1.0,5.0,227,look,500.0,487,87,0,polygon hybrid bicycle lightly bike balcony cl...,-1,0.152828,bike,size,carbon,shimano,frame,mm,road,new,cm,speed,0
4,1,7,WTS/WTT Scott Speedster 20 (Size 56),"2,000",Like new,Meetup,2 days ago,Road Bikes,Posted,Used once or twice. As good as new.,N,Y,Cashew MRT Station (DT2),rizeal,Joined 1 year ago,Very Responsive,Verified,1.0,0.0,1.0,5.0,10,Scott,2000.0,1237,186,0,wts wtt scott speedster 20 size 56 buy kh cycl...,-1,0.124765,bike,size,carbon,shimano,frame,mm,road,new,cm,speed,2
5,3,1,Giant TC ADVANCE 3 DISC,"1,600",Lightly used,Meetup,1 week ago,Road Bikes,Posted,"Used with care. Flaws, if any, are barely noti...",N,Y,Prestige heights,zhaoyichen82949,Joined 1 month ago,Not Responsive,Verified,1.0,0.0,1.0,0.0,0,Giant,1600.0,71,7,0,giant tc advance 3 disc 年初购入，因为要回，忍痛转让 尺寸m，附赠打...,1,0.543675,giant,tcr,shimano,advanced,pro,size,carbon,disc,wheelset,advance,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1430,18,4,Giant TCR Advance 105 (neg) carbon,880,Lightly used,Meetup,1 week ago,Road Bikes,Posted,"Used with care. Flaws, if any, are barely noti...",N,Y,Bustop opp Yishun ParkPhoto Booth @ Yishun MRT,kionzz,Joined 5 years ago,Very Responsive,Verified,NaN,NaN,NaN,5.0,48,"Giant, Giant, giant",880.0,320,55,0,giant tcr advance 105 neg carbon giant tcr adv...,1,1.000000,giant,tcr,shimano,advanced,pro,size,carbon,disc,wheelset,advance,1
1435,31,10,Force Etap Merida Scultura Bahrain Edition dis...,"2,300",Lightly used,Meetup,1 week ago,Road Bikes,Posted,"Used with care. Flaws, if any, are barely noti...",N,Y,Beauty World Centre,kcr_29,Joined 9 years ago,Very Responsive,Verified,NaN,NaN,NaN,5.0,130,"giant, pinarello, specialized, Merida",2300.0,449,68,0,force etap merida scultura bahrain edition dis...,-1,0.120732,bike,size,carbon,shimano,frame,mm,road,new,cm,speed,2
1436,15,5,Merida Scultura 5000 2022 - Size S - Green - F...,"2,000",Lightly used,Meetup,2 weeks ago,Road Bikes,Posted,"Used with care. Flaws, if any, are barely noti...",N,Y,Esso Upper East Coast Road,ernetlo,Joined 7 years ago,Mostly Responsive,Verified,NaN,NaN,NaN,5.0,29,Merida,2000.0,585,97,0,merida scultura 5000 2022 size s green framese...,2,1.000000,merida,carbon,reacto,bike,ultegra,shimano,size,scultura,frame,road,2
1440,17,1,Orbea Full carbon road bike frame with ultegra...,600,Well used,Meetup,Yesterday,Road Bikes,Posted,Has minor flaws or defects.,N,Y,Paya Lebar MRT Interchange (EW8

In [17]:
low_model_df = low_model_df.drop(columns=['topic', 'num_emojis'])

In [18]:
current_directory = os.getcwd()
csv_file_path = os.path.join(current_directory, '../data', 'final_for_pycaret.csv')

low_model_df.to_csv(csv_file_path, index=False)
